# 0. Library & Data

In [126]:
import pandas as pd
import json
import re
import os
import requests 
import zipfile 
import io 

from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report
from typing import Dict, Any, Callable, Tuple, List, Optional

import warnings
warnings.filterwarnings('ignore')

In [127]:
def load_data(url: str = 'data/') -> Tuple[pd.DataFrame, pd.DataFrame]:
    """훈련 및 테스트 데이터 파일을 로드하고 결합합니다. 파일이 없으면 GitHub에서 다운로드합니다."""
    print("1. 데이터 로드 시작...")
    
    # 필수 파일 존재 여부 확인
    # 파일 이름은 수정할 것
    required_files = ['train.csv', 'general_dialog1.csv', 'general_dialog2.csv', 'test.json']
    all_files_exist = all(os.path.exists(os.path.join(url, f)) for f in required_files)

    if not all_files_exist:
        print("필수 데이터 파일이 없습니다. GitHub에서 다운로드합니다...")
        download_url = "https://github.com/tunib-ai/DKTC/archive/refs/heads/main.zip"
        
        try:
            # data 디렉토리가 없으면 생성
            os.makedirs(url, exist_ok=True)
            
            print("데이터 다운로드 중...")
            response = requests.get(download_url)
            response.raise_for_status() # HTTP 오류 발생 시 예외 발생

            print("다운로드 완료, 압축 해제 중...")
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                for file in z.namelist():
                    # DKTC-main/data/ 폴더 안 파일만 추출
                    if file.startswith("DKTC-main/data/") and not file.endswith('/'):
                        # 파일 경로 재설정: 'DKTC-main/data/파일명' -> 'data/파일명'
                        target_path = os.path.join(url, os.path.relpath(file, "DKTC-main/data"))
                        os.makedirs(os.path.dirname(target_path), exist_ok=True)
                        with z.open(file) as source, open(target_path, "wb") as target:
                            target.write(source.read())
            print(f"'{url}' 폴더에 데이터 추출 완료!")
        except requests.exceptions.RequestException as e:
            print(f"데이터 다운로드 실패: HTTP 오류 또는 연결 문제. {e}")
            raise FileNotFoundError("데이터를 다운로드할 수 없습니다. 수동으로 파일을 넣어주세요.")
        except Exception as e:
            print(f"파일 압축 해제/저장 중 오류 발생: {e}")
            raise FileNotFoundError("다운로드 후 파일 저장에 실패했습니다.")


    # --- 데이터 로드 진행 (파일이 존재하거나 다운로드 후) ---
    train = pd.read_csv(os.path.join(url, 'train.csv'))
    a = pd.read_csv(os.path.join(url, 'general_dialog1.csv')).rename(columns={'dialogue': 'conversation'})
    a['class'] = '일반 대화'
    a['idx'] = range(0,len(a))
    a = a[['idx','class','conversation']]
    b = pd.read_csv(os.path.join(url, 'general_dialog2.csv'))
    b = b[b['class'] == "일반 대화"] 
    train_df = pd.concat([train, a, b], axis=0, ignore_index=True).drop(columns='idx', errors='ignore')

    # 테스트 데이터 로드 (test.json)
    with open(os.path.join(url, "test.json"), "r", encoding="utf-8") as f:
        test_df = pd.DataFrame({"conversation": [v["text"] for v in json.load(f).values()]})
    
    print(f"   - 훈련 데이터 shape: {train_df.shape}")
    print(f"   - 테스트 데이터 shape: {test_df.shape}")
    return train_df, test_df

# 1. Preprocessing & Encoding

In [128]:
def preprocess_data(train_df: pd.DataFrame, test_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, Dict[int, str]]:
    """텍스트 데이터를 정제하고 요청된 고정된 숫자 레이블로 인코딩합니다."""
    print("2. 데이터 전처리 시작...")
    
    # 전처리 함수: 특수 문자 및 다중 공백 제거 (생략)
    def clean_text(text: str) -> str:
        text = str(text) 
        text = re.sub(r'[^\w\s]', ' ', text, flags=re.UNICODE) 
        text = re.sub(r'\s+', ' ', text).strip()
        return text 

    train_df['cleaned_conversation'] = train_df['conversation'].apply(clean_text)
    test_df['cleaned_conversation'] = test_df['conversation'].apply(clean_text)

    LABEL_MAPPING = {
        '협박 대화': 0,
        '갈취 대화': 1,
        '직장 내 괴롭힘 대화': 2,
        '기타 괴롭힘 대화': 3,
        '일반 대화': 4
    }
    
    # 인코딩 수행
    train_df['label_encoded'] = train_df['class'].map(LABEL_MAPPING)
    
    print(f"   - 레이블 클래스: {len(LABEL_MAPPING)}개")
    return train_df, test_df

In [129]:
from konlpy.tag import Okt
import pandas as pd
from typing import List, Callable

# 전역 Okt 객체
okt_global = Okt()

# 전역 함수로 정의 (내부 함수 X)
def okt_tokenizer_morphs(text: str, stop_words: List[str] = None) -> List[str]:
    if stop_words is None:
        stop_words = []
    if pd.isna(text):
        return []
    tokens = okt_global.morphs(text, stem=True, norm=True)
    return [t for t in tokens if t not in stop_words and len(t) > 1]

def okt_tokenizer_nouns(text: str, stop_words: List[str] = None) -> List[str]:
    if stop_words is None:
        stop_words = []
    if pd.isna(text):
        return []
    tokens = okt_global.nouns(text)
    return [t for t in tokens if t not in stop_words and len(t) > 1]

def okt_tokenizer_pos(text: str, stop_words: List[str] = None) -> List[str]:
    if stop_words is None:
        stop_words = []
    if pd.isna(text):
        return []
    include_tags = {"Noun", "Verb", "Adjective", "Exclamation", "Adverb"}
    pos_tags = okt_global.pos(text, stem=True, norm=True)
    tokens = [
        word for word, tag in pos_tags
        if tag in include_tags and len(word) > 1 and word not in stop_words
    ]
    return tokens

# 매핑 딕셔너리
OKT_TOKENIZER_MAP = {
    'morphs': okt_tokenizer_morphs,
    'nouns': okt_tokenizer_nouns,
    'pos': okt_tokenizer_pos
}

In [130]:
def encode_data_tfidf(
    train_df: pd.DataFrame, 
    test_df: pd.DataFrame, 
    tokenizer_name: str, 
    okt_mode: Optional[str] = 'morphs',
    stop_words: List[str] = None
) -> Tuple[Any, Any, Any, Any, Any, Any]:
    """
    TF-IDF 인코딩 (캐시 없이 매번 새로 계산)
    """
    print("3. Tfidf Encoding 시작...")

    # ---------- 토크나이저 선택 ----------
    if tokenizer_name == 'okt':
        if okt_mode not in OKT_TOKENIZER_MAP:
            raise ValueError("okt_mode는 'morphs', 'nouns', 'pos' 중 하나여야 합니다.")
        base_tokenizer = OKT_TOKENIZER_MAP[okt_mode]
        print(f"   - Tfidf 토크나이저 사용: Okt (Mode: {okt_mode})")
    elif tokenizer_name == 'simple':
        base_tokenizer = simple_tokenizer
        print(f"   - Tfidf 토크나이저 사용: Simple (공백 분리)")
    else:
        raise ValueError(f"지원하지 않는 토크나이저: {tokenizer_name}")

    # stop_words 를 토크나이저에 전달
    def tokenizer_with_sw(text):
        return base_tokenizer(text, stop_words=stop_words or [])

    # ---------- TF-IDF ----------
    tfidf_vect = TfidfVectorizer(tokenizer=tokenizer_with_sw)

    X = train_df['cleaned_conversation']
    y = train_df['label_encoded']

    X_tfidf = tfidf_vect.fit_transform(X)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_tfidf, y, test_size=0.2, random_state=42, stratify=y
    )
    X_test_final = tfidf_vect.transform(test_df['cleaned_conversation'])

    print(f"   - **특징(컬럼) 개수 (Vocabulary Size): {X_tfidf.shape[1]}**")
    print(f"   - 학습용 데이터 shape: {X_tr.shape}")
    print(f"   - 검증용 데이터 shape: {X_val.shape}")

    return X_tr, X_val, y_tr, y_val, X_test_final, tfidf_vect

# 2. ML Train & F1-score

In [131]:
def train_and_evaluate_model(model_name: str, model_class: Callable, model_params: Dict[str, Any], 
                             X_tr, X_val, y_tr, y_val) -> Dict[str, Any]:
    """특정 모델로 학습을 진행하고 검증 데이터로 성능을 평가합니다. XGBoost CPU/GPU 설정 포함."""
    
    if model_name.startswith('XGBoost'):
        if 'tree_method' not in model_params:
                # 1. tree_method가 명시되지 않은 경우: 'hist'로 강제 설정 (XGBoostError 방지)
                model_params['tree_method'] = 'hist' 
                print("   - XGBoost: tree_method가 명시되지 않아 CPU 기반 'hist' 사용.")
        else:
                # 2. tree_method가 명시된 경우: 사용자가 설정한 값을 출력
                method = model_params['tree_method']
                if method == 'gpu_hist':
                    # 사용자가 명시적으로 'gpu_hist'를 설정하면 오류 가능성을 안내
                    print(f"   - XGBoost: GPU 가속 ('{method}')을 사용합니다. (GPU 미컴파일 시 오류 발생 가능)")
                elif method == 'hist':
                    print(f"   - XGBoost: CPU 기반 병렬 처리 ('{method}')을 사용합니다.")
                
        model_params.setdefault('use_label_encoder', False)
        model_params.setdefault('eval_metric', 'mlogloss')

            
    print(f"\n 모델 학습 및 평가: {model_name} 시작...")

    model = model_class(**model_params)
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val)
    
    f1_micro = f1_score(y_val, y_pred, average='micro')
    CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']
    report = classification_report(y_val, y_pred,target_names=CLASS_NAMES)
    
    print(f"   - F1 Micro Score: {f1_micro:.4f}")
    print("\n[ Classification Report ]")
    print(report)
    
    return {
        'model_name': model_name,
        'model': model,
        'f1_micro': f1_micro,
        'classification_report': report
    }

In [132]:
def run_experiment_flexible(
    data_path: str = './data/',
    tokenizer_name: str = 'okt',
    okt_mode: Optional[str] = 'morphs',
    experiments_config: List[Dict[str, Any]] = None,
    stop_words: List[str] = None
):
    print(" ML 파이프라인 실행 시작 ")
    try:
        # 1. 데이터 로드 & 전처리 (한 번만)
        train_df, test_df = load_data(url=data_path)
        train_df, test_df = preprocess_data(train_df, test_df)

        # 2. TF-IDF (캐시 없이)
        X_tr, X_val, y_tr, y_val, X_test_final, tfidf_vect = encode_data_tfidf(
            train_df, test_df,
            tokenizer_name=tokenizer_name,
            okt_mode=okt_mode,
            stop_words=stop_words
        )
    except Exception as e:
        print(f"\n데이터 처리 중 오류: {e}")
        import traceback, sys
        traceback.print_exc(file=sys.stdout)
        return None, None

    # ---------- 모델 실험 ----------
    if experiments_config is None:
        experiments_config = [
            {'name': 'LR_Baseline',  'class': LogisticRegression,
             'params': {'random_state': 24, 'solver': 'liblinear'}},
            {'name': 'RF_Baseline',  'class': RandomForestClassifier,
             'params': {'random_state': 24}},
            {'name': 'XGBoost_Baseline', 'class': XGBClassifier,
             'params': {'random_state': 24}}
        ]

    results = []
    for exp in experiments_config:
        result = train_and_evaluate_model(
            exp['name'], exp['class'], exp['params'].copy(),
            X_tr, X_val, y_tr, y_val
        )
        results.append(result)

    # ---------- 요약 ----------
    print("\n==============================================")
    print("최종 실험 결과 요약 (F1 Micro)")
    for r in results:
        print(f"- **{r['model_name']}**: **{r['f1_micro']:.4f}**")
    print("==============================================")

    return results, X_test_final

# 3. Submission 만들기

In [133]:
def make_submission(best_model: Any, X_test_final: Any, data_path: str = './data/', name='') -> None:
    
    print("\n🚀 Test 데이터 최종 추론 시작...")
    
    # Test 데이터 추론 (숫자 레이블 0, 1, 2, 3, 4)
    y_test_pred_encoded = best_model.predict(X_test_final)
    y_submission = y_test_pred_encoded # NumPy 배열 (추론 결과)

    # 1. 파일 경로 설정
    base_submission_path = os.path.join(data_path, 'submission.csv')
    
    # 2. [핵심 수정]: 기본 submission.csv 파일을 강제로 불러옵니다 (try-except 제거).
    submission_df = pd.read_csv(base_submission_path)
    print(f"   - 기존 Submission 양식 파일 로드: {base_submission_path} (Shape: {submission_df.shape})")

    # 3. 'class' 컬럼에 추론 결과 덮어쓰기
    if len(submission_df) == len(y_submission):
        submission_df['class'] = y_submission
    else:
        print(f"경고: 추론 결과({len(y_submission)})와 기존 제출 파일 행({len(submission_df)})의 개수가 불일치합니다. 강제로 class 컬럼을 덮어씁니다.")
        submission_df['class'] = y_submission 
        
    # 4. Submission 파일 저장 (name 포함)
    submission_path = os.path.join(data_path, f'submission{name}.csv')
    submission_df.to_csv(submission_path, index=False, encoding='utf-8-sig')
    
    print(f"최종 추론 완료 및 Submission 파일 저장: {submission_path} (Shape: {submission_df.shape})")

# 4. PipeLine & Submission

In [134]:
if __name__ == "__main__":
    # 실험 1
    stop_words = ["대리", "회사", "과장", "부장", "업무", "휴가", "퇴근", "자네"]
    mode = "pos"
    print("--- 1차 실험: ---")
    results1, X_test1 = run_experiment_flexible(tokenizer_name='okt', okt_mode=mode, stop_words=stop_words)
    best1 = max(results1, key=lambda x: x['f1_micro'])
    make_submission(best1['model'], X_test1, name='_pos')

    # 실험 2:
    # stop_words = []
    # mode = "pos"
    # print("--- 2차 실험: ---")
    # results1, X_test1 = run_experiment_flexible(tokenizer_name='okt', okt_mode=mode, stop_words=stop_words)
    # best1 = max(results1, key=lambda x: x['f1_micro'])
    # make_submission(best1['model'], X_test1, name='_pos')

--- 1차 실험: ---
 ML 파이프라인 실행 시작 
1. 데이터 로드 시작...
   - 훈련 데이터 shape: (5048, 2)
   - 테스트 데이터 shape: (500, 1)
2. 데이터 전처리 시작...
   - 레이블 클래스: 5개
3. Tfidf Encoding 시작...
   - Tfidf 토크나이저 사용: Okt (Mode: pos)
   - **특징(컬럼) 개수 (Vocabulary Size): 9552**
   - 학습용 데이터 shape: (4038, 9552)
   - 검증용 데이터 shape: (1010, 9552)

 모델 학습 및 평가: LR_Baseline 시작...
   - F1 Micro Score: 0.8515

[ Classification Report ]
              precision    recall  f1-score   support

       협박 대화       0.88      0.80      0.84       179
       갈취 대화       0.87      0.84      0.85       196
 직장 내 괴롭힘 대화       0.80      0.85      0.82       196
   기타 괴롭힘 대화       0.74      0.84      0.79       219
       일반 대화       1.00      0.92      0.96       220

    accuracy                           0.85      1010
   macro avg       0.86      0.85      0.85      1010
weighted avg       0.86      0.85      0.85      1010


 모델 학습 및 평가: RF_Baseline 시작...
   - F1 Micro Score: 0.8168

[ Classification Report ]
              precision    

# 이전 실험

In [95]:
if __name__ == "__main__":
    
    # 🧪 실험 1: Okt (morphs) 사용 - 기본 설정
    print("--- 1차 실험: Okt  사용 ---")
    results_morphs, X_test_final_morphs = run_experiment_flexible( 
        tokenizer_name='okt',
        okt_mode='pos'
    )

    # 최고 성능 모델 선택
    best_result = max(results_morphs, key=lambda x: x['f1_micro'])
    best_model = best_result['model']
    best_model_name = best_result['model_name']
    
    print("\n==============================================")
    print(f"최적 모델 선정: {best_model_name} (F1 Micro: {best_result['f1_micro']:.4f})")
    
    # 2. make_submission 호출
    make_submission(
        best_model=best_model, 
        X_test_final=X_test_final_morphs, 
        #data_path=data_path,
        name='_baseline'
    )

--- 1차 실험: Okt  사용 ---
 ML 파이프라인 실행 시작 
1. 데이터 로드 시작...
   - 훈련 데이터 shape: (5048, 2)
   - 테스트 데이터 shape: (500, 1)
2. 데이터 전처리 시작...
   - 레이블 클래스: 5개
3. Tfidf Encoding 시작...
   - Tfidf 토크나이저 사용: Okt (Mode: pos)
   - **특징(컬럼) 개수 (Vocabulary Size): 9560**
   - 학습용 데이터 shape: (4038, 9560)
   - 검증용 데이터 shape: (1010, 9560)

 모델 학습 및 평가: LR_Baseline 시작...
   - F1 Micro Score: 0.8653

[ Classification Report ]
              precision    recall  f1-score   support

       협박 대화       0.88      0.81      0.84       179
       갈취 대화       0.87      0.83      0.85       196
 직장 내 괴롭힘 대화       0.87      0.87      0.87       196
   기타 괴롭힘 대화       0.75      0.88      0.81       219
       일반 대화       1.00      0.93      0.96       220

    accuracy                           0.87      1010
   macro avg       0.87      0.86      0.87      1010
weighted avg       0.87      0.87      0.87      1010


 모델 학습 및 평가: RF_Baseline 시작...
   - F1 Micro Score: 0.8396

[ Classification Report ]
              preci

In [96]:
if __name__ == "__main__":
    
    # 🧪 실험 2: Okt (nouns) 사용 - 기본 설정
    print("--- 2차 실험: Okt (nouns) 사용 ---")
    results_morphs, X_test_final_morphs = run_experiment_flexible( 
        tokenizer_name='okt',
        okt_mode='nouns'
    )

    # 최고 성능 모델 선택
    best_result = max(results_morphs, key=lambda x: x['f1_micro'])
    best_model = best_result['model']
    best_model_name = best_result['model_name']
    
    print("\n==============================================")
    print(f"최적 모델 선정: {best_model_name} (F1 Micro: {best_result['f1_micro']:.4f})")
    
    # 2. make_submission 호출
    make_submission(
        best_model=best_model, 
        X_test_final=X_test_final_morphs, 
        #data_path=data_path,
        name='_baseline_nouns'
    )

--- 2차 실험: Okt (nouns) 사용 ---
 ML 파이프라인 실행 시작 
1. 데이터 로드 시작...
   - 훈련 데이터 shape: (5048, 2)
   - 테스트 데이터 shape: (500, 1)
2. 데이터 전처리 시작...
   - 레이블 클래스: 5개
3. Tfidf Encoding 시작...
   - Tfidf 토크나이저 사용: Okt (Mode: nouns)
   - **특징(컬럼) 개수 (Vocabulary Size): 7867**
   - 학습용 데이터 shape: (4038, 7867)
   - 검증용 데이터 shape: (1010, 7867)

 모델 학습 및 평가: LR_Baseline 시작...
   - F1 Micro Score: 0.8099

[ Classification Report ]
              precision    recall  f1-score   support

       협박 대화       0.75      0.63      0.68       179
       갈취 대화       0.76      0.74      0.75       196
 직장 내 괴롭힘 대화       0.87      0.87      0.87       196
   기타 괴롭힘 대화       0.69      0.85      0.76       219
       일반 대화       1.00      0.92      0.96       220

    accuracy                           0.81      1010
   macro avg       0.81      0.80      0.81      1010
weighted avg       0.82      0.81      0.81      1010


 모델 학습 및 평가: RF_Baseline 시작...
   - F1 Micro Score: 0.7782

[ Classification Report ]
          

In [97]:
if __name__ == "__main__":
    
    # 🧪 실험 2: Okt (pos) 사용 - 기본 설정
    print("--- 3차 실험: Okt (pos) 사용 ---")
    results_morphs, X_test_final_morphs = run_experiment_flexible( 
        tokenizer_name='okt',
        okt_mode='morphs'
    )

    # 최고 성능 모델 선택
    best_result = max(results_morphs, key=lambda x: x['f1_micro'])
    best_model = best_result['model']
    best_model_name = best_result['model_name']
    
    print("\n==============================================")
    print(f"최적 모델 선정: {best_model_name} (F1 Micro: {best_result['f1_micro']:.4f})")
    
    # 2. make_submission 호출
    make_submission(
        best_model=best_model, 
        X_test_final=X_test_final_morphs, 
        #data_path=data_path,
        name='_baseline_pos'
    )

--- 3차 실험: Okt (pos) 사용 ---
 ML 파이프라인 실행 시작 
1. 데이터 로드 시작...
   - 훈련 데이터 shape: (5048, 2)
   - 테스트 데이터 shape: (500, 1)
2. 데이터 전처리 시작...
   - 레이블 클래스: 5개
3. Tfidf Encoding 시작...
   - Tfidf 토크나이저 사용: Okt (Mode: morphs)
   - **특징(컬럼) 개수 (Vocabulary Size): 10110**
   - 학습용 데이터 shape: (4038, 10110)
   - 검증용 데이터 shape: (1010, 10110)

 모델 학습 및 평가: LR_Baseline 시작...
   - F1 Micro Score: 0.8733

[ Classification Report ]
              precision    recall  f1-score   support

       협박 대화       0.87      0.82      0.84       179
       갈취 대화       0.87      0.86      0.87       196
 직장 내 괴롭힘 대화       0.89      0.87      0.88       196
   기타 괴롭힘 대화       0.76      0.88      0.82       219
       일반 대화       1.00      0.93      0.96       220

    accuracy                           0.87      1010
   macro avg       0.88      0.87      0.87      1010
weighted avg       0.88      0.87      0.87      1010


 모델 학습 및 평가: RF_Baseline 시작...
   - F1 Micro Score: 0.8337

[ Classification Report ]
        

# 회고록 

- 일반 baselind에서 Okt 변화
    - 아무래도 그냥 명사만 뽑냐, 그 외 전부 사용하냐에 따라서 약간의 차이가 있는 듯 하다.
    - 명사만 뽑게 될 경우, 실제 행위나 그런 동작들을 알 수가 없기 때문에 그런 듯 하다.
        - morphs : 최적 모델 선정: LR_Baseline (F1 Micro: 0.8881)
        - nouns : 최적 모델 선정: LR_Baseline (F1 Micro: 0.8446)
        - pos: 최적 모델 선정: LR_Baseline (F1 Micro: 0.8653)
            - {"Noun", "Verb", "Adjective", "Exclamation", "Adverb"}만 추출해봄